## Make the dataframe including the drought area for the each years and regions
- Date : Mar 20, 2023
- Author : JaeEun Yoo

In [1]:
import os
import glob
import pandas as pd
import shutil
import re

In [6]:
root_path = '../샘플데이터/03. 통계청-가뭄피해면적/'
file_list = glob.glob(root_path+'/*')
file_list.sort()

In [7]:
file_list

['../샘플데이터/03. 통계청-가뭄피해면적/가뭄정보통계(피해 결과표2021).xlsx',
 '../샘플데이터/03. 통계청-가뭄피해면적/통계청)가뭄정보통계(피해 결과표2018).xlsx',
 '../샘플데이터/03. 통계청-가뭄피해면적/통계청)가뭄정보통계(피해 결과표2019).xlsx',
 '../샘플데이터/03. 통계청-가뭄피해면적/통계청)가뭄정보통계(피해 결과표2020).xlsx']

In [206]:
year = '2018'
columns_list = ['Si','SiGunGu','Paddy_StartDate','Paddy_EndDate','Paddy_drght',
                'Field_StartDate','Field_EndDate','Field_drght']

In [207]:
# Get the sheet names
data = pd.read_excel(file_list[1],header=3,sheet_name=None)
sheet_names = [k for k in data.keys()]
sheet_names

['2. 피해']

In [237]:
data = pd.read_excel(file_list[1],header=6,sheet_name=sheet_names[0])
df = data[data.keys()[1:3]]
df = pd.concat([df,data[data.keys()[20:]].reset_index(drop=True)],axis=1)
# 2021 : 354
# 2020 : 254
# 2018 : 272
df = df[2:272]
del df['전체 논 재배면적 (ha)']
del df['전체 밭 재배면적 (ha)']
df = df.reset_index(drop=True)

In [238]:
df.columns = columns_list

In [234]:
si = ''
for d in range(len(df)):
    # Remove the 'SUMS'
    if df['SiGunGu'][d] == '소계':
        df['Si'][d+1] = df['Si'][d]
    # Fill the 'Si' field
    elif pd.isna(df['Si'][d]):
        df['Si'][d] = df['Si'][d-1]
df = df.fillna('-')

sogye = df[df['SiGunGu'].str.contains('소계')].index
df = df.drop(sogye)

df.to_csv('../results_data/03_가뭄피해면적_'+year+'.csv',index=False,encoding='cp949')